In [1]:
import numpy as np
import cv2 as cv
import glob
import os

# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((5*8,3), np.float32)
objp[:,:2] = np.mgrid[0:8,0:5].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

images = glob.glob('./car/*.jpg')
print(f"Found {len(images)} .jpg images")

for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (8,5), None)

    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        print(f"✅ Chessboard found in {fname}")

        corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners2)

        # Draw and display the corners
        cv.drawChessboardCorners(img, (8,5), corners2, ret)
        cv.imshow('img', img)
        cv.waitKey(500)

cv.destroyAllWindows()

Found 12 .jpg images
✅ Chessboard found in ./car/123_90_57f2a0b0-adce-11f0-b2a9-a46bb60726e6.jpg
✅ Chessboard found in ./car/139_90_87a72952-adce-11f0-b2a9-a46bb60726e6.jpg
✅ Chessboard found in ./car/114_80_91d9e658-adce-11f0-b2a9-a46bb60726e6 - Copy.jpg
✅ Chessboard found in ./car/190_154_93c086ac-adce-11f0-b2a9-a46bb60726e6.jpg
✅ Chessboard found in ./car/139_81_907e1df6-adce-11f0-b2a9-a46bb60726e6.jpg
✅ Chessboard found in ./car/114_80_91d9e658-adce-11f0-b2a9-a46bb60726e6.jpg
✅ Chessboard found in ./car/92_73_8a0bec96-adce-11f0-b2a9-a46bb60726e6.jpg
✅ Chessboard found in ./car/147_111_9ed2d9a0-adce-11f0-b2a9-a46bb60726e6.jpg
✅ Chessboard found in ./car/155_146_84894138-adce-11f0-b2a9-a46bb60726e6.jpg
✅ Chessboard found in ./car/166_115_8b41c69e-adce-11f0-b2a9-a46bb60726e6.jpg
✅ Chessboard found in ./car/139_90_87a72952-adce-11f0-b2a9-a46bb60726e6 - Copy.jpg


In [2]:
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

In [3]:
input_folder = './car'
output_folder = './output'
os.makedirs(output_folder, exist_ok=True)

In [4]:
images = glob.glob(os.path.join(input_folder, '*.jpg'))
print(f"Found {len(images)} images in '{input_folder}'")

Found 12 images in './car'


In [5]:
# undistort
for i, fname in enumerate(images, 1):
    img = cv.imread(fname)
    if img is None:
        print(f"❌ Could not read {fname}")
        continue

    h, w = img.shape[:2]
    newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

    # Undistort
    dst = cv.undistort(img, mtx, dist, None, newcameramtx)

    # Optional crop
    x, y, w, h = roi
    dst = dst[y:y+h, x:x+w]

    # Save output image
    basename = os.path.basename(fname)
    out_path = os.path.join(output_folder, f"undistorted_{basename}")
    cv.imwrite(out_path, dst)
    print(f"[{i}/{len(images)}] ✅ Saved {out_path}")


[1/12] ✅ Saved ./output/undistorted_80_122_4cf9989e-adce-11f0-b2a9-a46bb60726e6.jpg
[2/12] ✅ Saved ./output/undistorted_123_90_57f2a0b0-adce-11f0-b2a9-a46bb60726e6.jpg
[3/12] ✅ Saved ./output/undistorted_139_90_87a72952-adce-11f0-b2a9-a46bb60726e6.jpg
[4/12] ✅ Saved ./output/undistorted_114_80_91d9e658-adce-11f0-b2a9-a46bb60726e6 - Copy.jpg
[5/12] ✅ Saved ./output/undistorted_190_154_93c086ac-adce-11f0-b2a9-a46bb60726e6.jpg
[6/12] ✅ Saved ./output/undistorted_139_81_907e1df6-adce-11f0-b2a9-a46bb60726e6.jpg
[7/12] ✅ Saved ./output/undistorted_114_80_91d9e658-adce-11f0-b2a9-a46bb60726e6.jpg
[8/12] ✅ Saved ./output/undistorted_92_73_8a0bec96-adce-11f0-b2a9-a46bb60726e6.jpg
[9/12] ✅ Saved ./output/undistorted_147_111_9ed2d9a0-adce-11f0-b2a9-a46bb60726e6.jpg
[10/12] ✅ Saved ./output/undistorted_155_146_84894138-adce-11f0-b2a9-a46bb60726e6.jpg
[11/12] ✅ Saved ./output/undistorted_166_115_8b41c69e-adce-11f0-b2a9-a46bb60726e6.jpg
[12/12] ✅ Saved ./output/undistorted_139_90_87a72952-adce-11f0-b

In [6]:
print("ret =", ret)
print("Camera matrix (mtx):\n", mtx)
print("Distortion coefficients (dist):\n", dist)


ret = 5.005190370213506
Camera matrix (mtx):
 [[103.7775004    0.          97.94110885]
 [  0.         184.25810131  60.21977861]
 [  0.           0.           1.        ]]
Distortion coefficients (dist):
 [[-0.53971683  0.13060682  0.11503791  0.00822575  0.0036368 ]]
